In [1]:
# Import necessary libraries
import pandas as pd
import plotly.graph_objects as go
import dalmatian
from google.cloud import storage
import matplotlib.pyplot as plt
import networkx as nx
from networkx.drawing.nx_pydot import graphviz_layout

import numpy as np

import sys
import os
sys.path.append('..')  
import model_helper 
from helper import *
from plotly_helper import *

In [2]:
# Function to load data
def load_data(patient_id, wbc_file, treatment_file):
    # Load WBC and treatment data
    wbc_df = pd.read_csv(wbc_file)
    treatment_df = pd.read_csv(treatment_file, sep='\t')
    
    # Get input files from Terra through dalmatian
    workspace = 'broad-firecloud-ibmwatson/TAG_CLL_Clonal_Kinetic_UMI_PrAN'
    wm = dalmatian.WorkspaceManager(workspace)
    participants = wm.get_participants()
    
    # Load additional data for the patient
    cluster_CCF_df = pd.read_csv(participants.loc[patient_id]['cluster_ccfs'], sep='\t')
    abundance_df = pd.read_csv(participants.loc[patient_id]['abundances_tsv'], sep='\t')
    mcmc_df = pd.read_csv(participants.loc[patient_id]['cell_population_mcmc_trace'], sep='\t')
    tree_df = pd.read_csv(participants.loc[patient_id]['tree_tsv'], sep='\t')
    
    return wbc_df, treatment_df, cluster_CCF_df, abundance_df, mcmc_df, tree_df

# Function to filter data for a specific patient
def filter_patient_data(wbc_df, patient_id):
    wbc_df_patient = wbc_df[wbc_df['Patient'] == patient_id]
    wbc_df_patient.reset_index(drop=True, inplace=True)
    times_sample = [int(i) for i in wbc_df_patient.loc[wbc_df_patient.Sample.notna(), 'Day'].tolist()]
    ALC_all = [float(i) for i in wbc_df_patient['ALC'].tolist()]
    
    
    ALC_count_sample = [float(i) for i in wbc_df_patient.loc[wbc_df_patient.Sample.notna(), 'ALC'].tolist()]
    all_times = [int(i) for i in wbc_df_patient['Day'].to_list()]
    
    return wbc_df_patient, times_sample, ALC_all,ALC_count_sample,all_times





In [3]:
# Input patient information
# Input patient information
patient_id = 'RP-1895_FI-01'
wbc_file = '../combined_FI_alc.csv'
treatment_file = '../GCLL_treatment.txt'
UMI_start = 7
UMI_end = 8
treatment_start = 0


In [42]:
# Manually verify the sample list order is correct

In [4]:

sample_list = ['RP-1895_8881-C1D1_v1_Exome_OnPrem-RP-1895_273-SAL_v1_Exome_OnPrem', 'RP-1895_8974-C2D28_v1_Exome_OnPrem-RP-1895_273-SAL_v1_Exome_OnPrem', 'RP-1895_9072-C3D28_v1_Exome_OnPrem-RP-1895_273-SAL_v1_Exome_OnPrem', 'RP-1895_9181-C4D28_v1_Exome_OnPrem-RP-1895_273-SAL_v1_Exome_OnPrem', 'RP-1895_9383-C6D28_v1_Exome_OnPrem-RP-1895_273-SAL_v1_Exome_OnPrem', 'SM-N57VF_RP-1895_273-SAL_v1_Exome_OnPrem_pair', 'SM-N57VG_RP-1895_273-SAL_v1_Exome_OnPrem_pair', 'SM-N57VC'] 

In [44]:
# Output file

In [5]:
outputfile_name = "FI_subplot_combined_01_wbc_updated.html"

Update patient specific information in the below cell 

In [6]:
# Get the treatment duration 

treatment_end = 2359

In [7]:
treatment_end

2359

In [8]:

wbc_df, treatment_df, cluster_CCF_df, abundance_df, mcmc_df, tree_df = load_data(patient_id, wbc_file,treatment_file )

# Filter data for the specific patient
wbc_df_patient, times_sample,alc_count,alc_count_sample, all_times = filter_patient_data(wbc_df, patient_id)

# Plot CLL count over time and save as HTML
CLL_plot_html = plot_CLL_count(patient_id, times_sample, alc_count_sample, UMI_start, UMI_end, treatment_start, treatment_end, )
    




In [113]:
times_sample

[-1, 60, 97, 118, 181, 356, 706, 1441]

In [114]:
alc_count_sample

[247.06, 170.91, 60.3, 10.4, 5.03, 1.92, 1.38, 1.1]

In [115]:
wbc_table_html = plot_metadata_table(wbc_df_patient, patient_id )

In [116]:
tree_df

,n_iter,likelihood,edges
0,249,"[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, ...","1-2,None-1"


In [117]:
 plot_ccf_html = plot_ccf(cluster_CCF_df, times_sample, treatment_df,)

In [91]:
branch_annotations = {
    1: "MAP2K2",
    
    5:"ATM"
    
}

In [118]:
ccf_tree_html = plot_ccf_tree_combined(
    tree_df=tree_df,
    tree_selected=0
    ,
    ccf_df=cluster_CCF_df,
    times_sample=times_sample,
    treatment_df=treatment_df,
    branch_annotations=branch_annotations

)

In [119]:
cluster_list, cluster_abundance = model_helper.get_abundance(abundance_df, mcmc_df, sample_list)
subclone_sample, log_subclone_sample = model_helper.calc_subclone(alc_count_sample, cluster_abundance, cluster_list)
all_abundance = model_helper.get_all_abundance(cluster_list, mcmc_df, sample_list, times_sample, )

In [120]:
#  Add noise to all abundance

import random 
random.seed(42)
def add_uniform_noise_and_normalize(lst, low=0, high=0.01):
    
    # Add uniform noise
    noisy_lst = [x + random.uniform(low, high) for x in lst]

    
    # Normalize the list
    total_sum = sum(noisy_lst)
    
    normalized_lst = [x / total_sum for x in noisy_lst]

    return normalized_lst


subclone_cluster_iter = {}
# Iterate through each cluster
for cluster, iterations in all_abundance.items():
   
    # Iterate through each iteration in the cluster
    for iteration, value in iterations.items():
        # If the iteration is not in the new dict, add it
        if iteration not in subclone_cluster_iter:
            subclone_cluster_iter[iteration] = {}
        # Add the cluster and its value to this iteration
        subclone_cluster_iter[iteration][cluster] = value

        
        
noise_added_iter = {}

for iteration, abundances in subclone_cluster_iter.items():
    
    transposed_data = {i: list(values) for i, values in enumerate(zip(*abundances.values()), start=1)}
    
    noise_added_iter[iteration] = {}
    for time_point, abundance in transposed_data.items():
        
#         print(time_point, abundance)
        noise_abundance = add_uniform_noise_and_normalize(abundance)
    
#         print(noise_abundance)

        noise_added_iter[iteration][time_point] = add_uniform_noise_and_normalize(abundance)

    
noise_added_all_abundance = {}
for iteration, abundances in noise_added_iter.items():
    original_format = {i + 1: list(values) for i, values in enumerate(zip(*abundances.values()))}
    noise_added_all_abundance[iteration] = original_format
    
    
noise_added_original_all_abundances = {}

for iteration, abundances in noise_added_all_abundance.items():
    
    for cluster, abundance_value in abundances.items():
        if cluster not in noise_added_original_all_abundances:
            noise_added_original_all_abundances[cluster] = {}
        noise_added_original_all_abundances[cluster][iteration] = abundance_value
            

In [121]:
subclone_sample_mcmc_with_uniform_noise, log_subclone_sample_mcmc_with_uniform_noise = model_helper.calc_subclone(alc_count_sample, noise_added_original_all_abundances, cluster_list, input_type = "mcmc")

In [122]:
times_aft_tx = [x for x in all_times if x > treatment_end]
times_aft_tx.insert(0, treatment_end)

# Set the extrapolate time after treatment
extrapolate_start_idx = 0

In [97]:
times_sample

[-1, 60, 97, 118, 181, 356, 706, 1441]

In [123]:
times_sample[extrapolate_start_idx:]

[-1, 60, 97, 118, 181, 356, 706, 1441]

In [124]:
# Generate Plotly plot
subclone_plot_html = plot_subclones(cluster_list, times_sample, alc_count_sample, log_subclone_sample, extrapolate_start_idx, times_aft_tx, treatment_df, treatment_end)


In [125]:
linear_model_mcmc_html = plot_linear_model_mcmc(cluster_list,times_sample,  alc_count_sample, log_subclone_sample_mcmc_with_uniform_noise,extrapolate_start_idx,times_aft_tx,treatment_df, treatment_end)  

In [101]:
# Convert the column "CLL count estm" to float 
wbc_df_patient['CLL count estm'] = wbc_df_patient['CLL count estm'].astype(float) 
times_sliced_aft = [int(i) for i in wbc_df_patient[wbc_df_patient['CLL count estm'] > 0]['Time_since_start_tx'].values  if int(i) > 0]

KeyError: 'CLL count estm'

In [31]:
# Needs to manually check times_sliced_aft

In [44]:
times_sliced_aft

[793, 884, 1067, 1223, 1388, 1582, 1783, 1874]

In [45]:
len(times_sliced_aft)

8

In [47]:
index_samples_model = slice(1,6)

# index_samples_model is the index of the samples after treatment. For instance, slice(1,7) means sample 2, 3, 4, 5,6, 7

X, y = create_inputs(times_sliced_aft, log_subclone_sample_mcmc_with_uniform_noise,2, index_samples_model, times_sample)



In [48]:
y

[None,
 None,
 None,
 array([-7.10682012, -1.06224207, -5.81586732, -8.18792788, -7.06963802,
        -6.06325704]),
 array([-6.94964917, -2.15129209, -8.98175261, -6.38289616, -7.58356162,
        -7.4881173 ]),
 array([-4.7107572 , -0.01709826, -6.18086787, -3.07880393, -5.41273341,
        -4.31999285]),
 array([-3.98699104,  0.52341795, -4.01744361, -2.39997502, -2.64624857,
        -3.21885145]),
 array([-3.84888932,  1.26411095, -3.32491797,  0.82207778, -0.26070171,
         0.33064238])]

In [49]:
wbc_aft_tx = wbc_df_patient[wbc_df_patient.Time_since_start_tx > treatment_end]['CLL count estm'].to_list()
wbc_model = [i for i in wbc_aft_tx if i > 0]

In [50]:
wbc_model

[0.002617, 0.003432, 0.005507, 0.352723, 0.120105, 1.057253, 1.924824, 8.03]

In [51]:
len(wbc_model)

8

In [52]:
n_clusters = len(cluster_list)
logsumexp_points = np.log(wbc_model)
model = MultiClusterLinearRegression(n_clusters, X, y)
model.fit(logsumexp_points)

In [53]:
# Generate Plotly plot
subclone_plot_new_model_html = plot_subclones_new_model(cluster_list, times_sample, wbc_model, log_subclone_sample, extrapolate_start_idx, times_aft_tx, times_sliced_aft, treatment_df,treatment_end, model )


In [54]:
plot_mcmc_new_model_html = plot_mcmc_model(cluster_list, index_samples_model, times_aft_tx, times_sample, times_sliced_aft, sample_list, wbc_model, log_subclone_sample_mcmc_with_uniform_noise, treatment_df, treatment_end)


In [126]:
create_html_file([wbc_table_html, plot_ccf_html, ccf_tree_html, CLL_plot_html, subclone_plot_html,linear_model_mcmc_html,], output_file=outputfile_name)

HTML file saved as FI_subplot_combined_01_wbc_updated.html


In [102]:
create_html_file([wbc_table_html, plot_ccf_html, ccf_tree_html, CLL_plot_html, subclone_plot_html,linear_model_mcmc_html,subclone_plot_new_model_html, plot_mcmc_new_model_html], output_file=outputfile_name)

NameError: name 'subclone_plot_new_model_html' is not defined